## Carregando Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv

## Carregando Dados do Projeto

In [2]:
alunos = 'alunos.csv'
indicadores = 'indicadores.csv'
roi = 'roi.json.csv'
# Funcao para carregar dados e criar DataFrame
def csv_to_df(address):
    with open(address, 'r', encoding='utf-8') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        linhas = []
        inicio = 0
        dado_csv_json = []
        for linha in leitor_csv:
            if(inicio==0):
                linhas = linha
                inicio=1
            else:
                bloco = {coluna: '' for coluna in linhas}
                for x in range(len(linha)):
                    bloco[linhas[x]] = linha[x]
                dado_csv_json.append(bloco)
    df_dado = pd.DataFrame(dado_csv_json)
    return df_dado

df_alu = csv_to_df(alunos)
df_ind = csv_to_df(indicadores)
df_roi = csv_to_df(roi)

## Funções para Transformar cursos adicionais de alunos em linhas individuais

In [3]:

def remove_div(col):
    final = []
    for x in col:
        final.append(x.split('/')[-1])
    return final
def nova_linha(linha):
    vetor_v= [27,40,54,68,82,96,110,124,138,152,166,180,194,208,222] # vetor para verificar se usuario esta cursando mais de um curso
    vetor_n= [26,27,28,29,30,31,32,33,34,35,36,37,38,39] # posicoes do segundo curso, para aumentar adicione +n*14 onde n é o numero do curso do usuario
    vetor_n = np.array(vetor_n)
    cont=0
    # verifica a quantidade de cursos a mais a linha tem
    for x in vetor_v:
        if(len(linha.iloc[int(x)])):
            cont+=1
    #primeiro curso:
    vetor_f = linha.iloc[14:26] # parametro de usuario imutavel
    vetor_i = linha.iloc[0:14]
    cref = remove_div(vetor_i.index)
    vetor_i.index = cref
    completo = pd.concat([vetor_i,vetor_f])
    final = []
    idx = completo.index
    final.append(completo.values)
    for x in range(cont): 
        novo = vetor_i
        novo.index = cref
        corte = vetor_n+14*x
        rcorte = linha.iloc[corte]
        coluna = rcorte.index
        coluna = remove_div(coluna)
        rcorte.index = coluna
        for z in rcorte.index:
            novo[z] = rcorte[z]
        completo = pd.concat([novo,vetor_f])
        final.append(completo.values)
    idx = ['situacao', 'atualizadoEm', 'dataTermino', 'frequencia', 'ultimoLogin',
       'nome', 'frequenciaAprovacao', 'mediaAprovacao', 'idUnico',
       'iniciado', 'edital', 'dataInicio', 'frequenciaAtualizadaEm', 'nota',
       'status', 'remessa_id', 'registration_date', 'GSI1SK', 'name', 'GSI1PK',
       'profissional', 'number', 'identificadorUnico', 'sk', 'id', 'empresa']
    pd_final = pd.DataFrame(final,columns=idx)
    return pd_final


In [4]:
df_alu_formatado = pd.DataFrame()
for x in range(len(df_alu)):
    linha = df_alu.iloc[x]
    nova = nova_linha(linha)
    df_alu_formatado = pd.concat([df_alu_formatado,nova], axis=0)

### DataFrame de Alunos reduzido em suas colunas, alunos que se matricularam em mais de um curso tiveram uma adicao de linha no dataframe com as informacoes de cada curso matriculado

In [14]:
df_alu_formatado

,situacao,atualizadoEm,dataTermino,frequencia,ultimoLogin,nome,frequenciaAprovacao,mediaAprovacao,idUnico,iniciado,edital,dataInicio,frequenciaAtualizadaEm,nota,status,remessa_id,registration_date,GSI1SK,name,GSI1PK,profissional,number,identificadorUnico,sk,id,empresa
0,REPROVADO,1743190979435,2025-03-31T23:59:59.999Z,0,2024-12-03T17:04:22.719Z,Ia para educação,75,6,736657508,false,ED4/24C36T2,2024-12-03T00:00:00.000Z,1743103073510,0,COMPLETE,ON-DEMAND,2024-10-16T18:04:20.675Z,2nWvtQz5H0lBLmZXsaGtFxccnOc,ILANE KARISE BARBOSA CUNHA,STUDENT,true,+558598158447,276365396,STUDENT,2nWvtQz5H0lBLmZXsaGtFxccnOc,ENSINO DIGITAL
0,CURSANDO,1744807287593,2025-04-29T23:59:59.999Z,80,2025-04-15T13:43:35.864Z,Metodologias Ágeis para Desenvolvimento de Sof...,75,6,736657529,true,ED5/24C40T2,2025-02-26T00:00:00.000Z,1744728103051,8,COMPLETE,ON-DEMAND,2025-03-10T13:33:41.556Z,2u7xscxp2Cf9DfwbZtg525JHwQN,WILLIAM XAVIER DOS SANTOS,STUDENT,false,+556192523253,272662909,STUDENT,2u7xscxp2Cf9DfwbZtg525JHwQN,ENSINO DIGITAL
0,AUSENTE,,2024-08-20T23:59:59.999Z,0,2024-07-22T08:13:42.514Z,Power BI para Educação,,,736657412,false,,2024-07-22T11:13:04.199Z,,,COMPLETE,ON-DEMAND,2024-07-22T11:13:04.199Z,2jbDGAW98GYCTr0CATxmQH6DZ9B,DAVI GARCIA BEZERRA,STUDENT,,+558599997431,735969834,STUDENT,2jbDGAW98GYCTr0CATxmQH6DZ9B,ENSINO DIGITAL
1,REPROVADO,,2024-08-20T23:59:59.999Z,0,2024-07-22T08:13:42.514Z,Ia para educação,,,736657414,false,,2024-07-22T11:13:04.199Z,,,COMPLETE,ON-DEMAND,2024-07-22T11:13:04.199Z,2jbDGAW98GYCTr0CATxmQH6DZ9B,DAVI GARCIA BEZERRA,STUDENT,,+558599997431,735969834,STUDENT,2jbDGAW98GYCTr0CATxmQH6DZ9B,ENSINO DIGITAL
0,CURSANDO,1744809165564,2025-04-29T23:59:59.999Z,0,2025-04-04T23:01:06.886Z,Metodologias Ágeis para Desenvolvimento de Sof...,75,6,743768073,false,ED5/24C40T3,2025-02-27T00:00:00.000Z,1743196914477,0,COMPLETE,ON-DEMAND,2025-03-10T14:11:12.325Z,2u82RT5R8DEEIga9AEVUUNMufkX,GABRIELLE FERREIRA ROLIM,STUDENT,false,+558588408040,709133461,STUDENT,2u82RT5R8DEEIga9AEVUUNMufkX,ENSINO DIGITAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,CURSANDO,1744807264672,2025-04-29T23:59:59.999Z,60,2025-04-15T01:51:19.771Z,Introdução à Acessibilidade na Web,75,6,736657526,true,ED5/24C14T2,2025-02-10T00:00:00.000Z,1744727947774,4.400000095367432,COMPLETE,ON-DEMAND,2025-03-10T13:28:12.393Z,2u7xDKIqbCTxVzi9vGxFOewKJ9R,ELIZABETH LOPES GUEDES,STUDENT,false,+558596117562,272138310,STUDENT,2u7xDKIqbCTxVzi9vGxFOewKJ9R,ENSINO DIGITAL
0,REPROVADO,1738022071265,2025-01-14T23:59:59.999Z,25,2024-10-23T10:58:07.487Z,Ia para educação,75,6,736657497,true,ED3/24C36T2,2024-10-10T00:00:00.000Z,1729746642317,2.5,COMPLETE,ON-DEMAND,2024-10-10T21:06:09.748Z,2nGLGR8Ky6V9Exk5O3FQyxTtwDv,JOÃO CLÁUDIO SANT ANNA ESTEFANIO,STUDENT,false,+552164082648,735972815,STUDENT,2nGLGR8Ky6V9Exk5O3FQyxTtwDv,ENSINO DIGITAL
0,APROVADO,1743191281947,2025-03-31T23:59:59.999Z,100,2025-03-28T15:54:29.869Z,Ia para educação,75,6,736657518,true,ED4/24C36T3,2024-12-19T00:00:00.000Z,1743103377238,9.5,COMPLETE,ON-DEMAND,2024-10-23T05:00:16.730Z,2npAPA5Ud6DaWYFIsh8gUQkwtAz,JOSE DAVILA BARROS DOS REIS,STUDENT,true,+558586796104,272664718,STUDENT,2npAPA5Ud6DaWYFIsh8gUQkwtAz,ENSINO DIGITAL
1,CURSANDO,1744866123733,2025-04-29T23:59:59.999Z,23,2025-04-16T11:41:22.675Z,Introdução a Python,75,6,736657528,true,ED5/24C23T2,2025-02-11T00:00:00.000Z,1744866123733,2.299999952316284,COMPLETE,ON-DEMAND,2024-10-23T05:00:16.730Z,2npAPA5Ud6DaWYFIsh8gUQkwtAz,JOSE DAVILA BARROS DOS REIS,STUDENT,true,+558586796104,272664718,STUDENT,2npAPA5Ud6DaWYFIsh8gUQkwtAz,ENSINO DIGITAL


### Analise de Tabelas

In [6]:
def uni_count(df_completo):
    descricao = df_completo.describe(include='all')
    saida = descricao.loc[['count','unique']]
    return saida
pd.set_option('display.max_columns', None)

#### Primeiramente iremos identificar colunas com apenas uma unica variavel se repetindo, essas colunas podem ser descartadas ja que não agrega para a distinção em classificadores

In [7]:
ddf_alu = uni_count(df_alu_formatado)
ddf_alu

,situacao,atualizadoEm,dataTermino,frequencia,ultimoLogin,nome,frequenciaAprovacao,mediaAprovacao,idUnico,iniciado,edital,dataInicio,frequenciaAtualizadaEm,nota,status,remessa_id,registration_date,GSI1SK,name,GSI1PK,profissional,number,identificadorUnico,sk,id,empresa
count,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949,7949
unique,5,4198,26,35,4975,14,2,2,137,2,55,1034,5172,120,1,1,4897,4897,4891,1,3,4864,4896,1,4897,1


Na linha de unique para o dataframe de alunos formatado, temos que as colunas com 1 no unico podem ser desconsideradas inicialmente

In [9]:
ddf_ind = uni_count(df_ind)
ddf_ind

,tipo,idAluno,data,chatId,nomeAluno,categoria,telefoneAluno,pk,empresa,interativo
count,10085,10085,10085,10085,10085,10085,10085,10085,10085,10085
unique,1,2623,10085,10085,2623,8,2932,10085,1,2


In [10]:
ddf_roi = uni_count(df_roi)
ddf_roi

,tipo,idAluno,data,chatId,nomeAluno,interativo,categoria,telefoneAluno,pk,empresa,efetivo
count,2100,2100,2100,2100,2100,2100,2100,2100,2100,2100,2100
unique,1,973,1894,1894,973,2,7,1036,1894,1,1


Verificando todas as colunas vemos que os idAluno de roi e indicadores são valores associados a identificadorUnico da tabela alunos que são associados a usuarios\
\
Analisando o conteudo para cada tabela de indicadores e roi podemos seguir para uma proposta de tabela com uma coluna para cada categoria distinta que aparece nas tabelas indicadores e roi, um layout final de planilha fica em |idAluno | interativo | cat1 | ... | catN | 
\
\
Na planilha de Aluno temos: sit1 | ... | sitN | 

In [34]:
def presenca(coluna1,coluna2):
    
    porcentagem = np.isin(coluna2, coluna1).mean() * 100
    print(f"{porcentagem:.2f}% dos elementos de {coluna2.name} estão em {coluna1.name}")
presenca(df_alu_formatado['identificadorUnico'],df_ind['idAluno'])
presenca(df_alu_formatado['identificadorUnico'],df_roi['idAluno'])
presenca(df_ind['idAluno'],df_roi['idAluno'])

99.97% dos elementos de idAluno estão em identificadorUnico
100.00% dos elementos de idAluno estão em identificadorUnico
100.00% dos elementos de idAluno estão em idAluno


In [26]:
df_ind

,tipo,idAluno,data,chatId,nomeAluno,categoria,telefoneAluno,pk,empresa,interativo
0,TOTAL_MENSAGENS_ENVIADAS,735975300,1738974975204,e446f915-03b7-45e2-8a1c-eee54938c8ff,ÂNGELO MIGUEL SOARES DE CASTRO,FINALIZAÇÃO CURSO,+556198635806,1738974975204,ENSINO DIGITAL,
1,TOTAL_MENSAGENS_ENVIADAS,743637878,1744039355016,a6b0eb8f-26f9-408b-879f-0ae9d660c500,LAURO IGOR METZ,PRAZO E POUCO DESEMPENHO,+554196851824,1744039355016,ENSINO DIGITAL,true
2,TOTAL_MENSAGENS_ENVIADAS,743637139,1745150708604,ae86291b-1cb4-4bb9-93ef-efc6f4f9a36e,CALEB PAIVA RODRIGUES,PRAZO E POUCO DESEMPENHO,+558591858903,1745150708604,ENSINO DIGITAL,true
3,TOTAL_MENSAGENS_ENVIADAS,703037977,1745150650239,8669fc61-f670-4147-8807-e1f25c0e30e2,BRUNO GABRIEL GUSMÃO FERRAZ CAVALCANTI,PRAZO E POUCO DESEMPENHO,+558199839590,1745150650239,ENSINO DIGITAL,true
4,TOTAL_MENSAGENS_ENVIADAS,743637147,1745150648185,2f456a97-d486-4325-bb30-d3ce94779051,DANILO PEREIRA DA SILVA,PRAZO E POUCO DESEMPENHO,+555519953395,1745150648185,ENSINO DIGITAL,true
...,...,...,...,...,...,...,...,...,...,...
10080,TOTAL_MENSAGENS_ENVIADAS,743637552,1744639106177,dd92a600-e7f6-43af-9f4f-f41b80354c2b,EMANUEL PEREIRA DE ARAUJO,FINALIZAÇÃO CURSO,+558487076092,1744639106177,ENSINO DIGITAL,true
10081,TOTAL_MENSAGENS_ENVIADAS,735975850,1744639125680,d4c2c078-59d3-4558-b042-2d6770d0f9bd,CLAUDIA REBOUÇAS LIMA FERNANDES,FINALIZAÇÃO CURSO,+558587998224,1744639125680,ENSINO DIGITAL,true
10082,TOTAL_MENSAGENS_ENVIADAS,703037527,1742591434160,4789773b-dbbf-4cb3-ba46-8bedbf21f70d,FRANCISCO ALEXANDRE GERMANO,NÃO INICIOU / 3º CONTATO,+558589681702,1742591434160,ENSINO DIGITAL,true
10083,TOTAL_MENSAGENS_ENVIADAS,703038221,1741608942764,326e7e1d-b264-4ef9-a5be-75e4059feccb,MARCELO AGOSTINHO DA COSTA PINTO,FINALIZAÇÃO CURSO,+553598064653,1741608942764,ENSINO DIGITAL,true


In [27]:
df_roi

,tipo,idAluno,data,chatId,nomeAluno,interativo,categoria,telefoneAluno,pk,empresa,efetivo
0,TOTAL_MENSAGENS_ENVIADAS,272138310,1744639235711,6b3eec59-ba38-43e2-8528-b7b12191ced9,ELIZABETH LOPES GUEDES,false,FINALIZAÇÃO CURSO,+558596117562,1744639235711,ENSINO DIGITAL,true
1,TOTAL_MENSAGENS_ENVIADAS,272334954,1742040175207,bc6ff55e-4506-458c-85c7-897d4b3c132f,LAECIO FERREIRA DADA SILVA SOBRINHO,false,INSCRITO E NÃO COMEÇOU,+558588246490,1742040175207,ENSINO DIGITAL,true
2,TOTAL_MENSAGENS_ENVIADAS,272334954,1742591386159,94da8b55-6ce7-4476-9112-80a5e480aca1,LAECIO FERREIRA DADA SILVA SOBRINHO,false,NÃO INICIOU / 3º CONTATO,+558588246490,1742591386159,ENSINO DIGITAL,true
3,TOTAL_MENSAGENS_ENVIADAS,272334954,1742904175236,6add9d41-0af5-450d-ae54-0b55500efaf7,LAECIO FERREIRA DADA SILVA SOBRINHO,false,INSCRITO E NÃO COMEÇOU,+558588246490,1742904175236,ENSINO DIGITAL,true
4,TOTAL_MENSAGENS_ENVIADAS,272334954,1744039315545,66428a66-1c18-479d-adba-d161cd46cee2,LAECIO FERREIRA DADA SILVA SOBRINHO,false,PRAZO E POUCO DESEMPENHO,+558588246490,1744039315545,ENSINO DIGITAL,true
...,...,...,...,...,...,...,...,...,...,...,...
2095,TOTAL_MENSAGENS_ENVIADAS,745406495,1744632206509,04ea09e0-8b57-4cf8-997a-efcd300171c2,ARTUR ROLLEMBERG CAMBOIM GUEDES,false,ALUNO TORNOU-SE AUSENTE,+557991411279,1744632206509,ENSINO DIGITAL,true
2096,TOTAL_MENSAGENS_ENVIADAS,745406495,1744718605650,db2f759c-fd4c-48d7-b8dd-4db1e1dc29bd,ARTUR ROLLEMBERG CAMBOIM GUEDES,false,ALUNO TORNOU-SE AUSENTE,+557991411279,1744718605650,ENSINO DIGITAL,true
2097,TOTAL_MENSAGENS_ENVIADAS,745406495,1745064206435,a379dc09-e126-4d70-8e67-f24f11466ba7,ARTUR ROLLEMBERG CAMBOIM GUEDES,false,ALUNO À 10 SEM PROGRESSO,+557991411279,1745064206435,ENSINO DIGITAL,true
2098,TOTAL_MENSAGENS_ENVIADAS,745406515,1744718606472,cf0a52a9-162f-4d11-894a-d730009540ff,HUTHIERRE BIBIANO DE OLIVEIRA,false,ALUNO TORNOU-SE AUSENTE,+555195746376,1744718606472,ENSINO DIGITAL,true


### Redução de Tabela Alunos e criação de nova tabela associando identificadorUnico + iterações de tabelas roi e indicadores 

In [50]:
colunas_para_remover = ddf_alu.columns[ddf_alu.loc['unique'] == 1]
df_alu_reduzida =  df_alu_formatado.drop(columns=colunas_para_remover)
df_alu_reduzida = df_alu_reduzida.drop(['GSI1SK','idUnico','atualizadoEm','dataTermino','ultimoLogin','edital','dataInicio','frequenciaAtualizadaEm','registration_date','name','number','id'], axis=1)
df_alu_reduzida

,situacao,frequencia,nome,frequenciaAprovacao,mediaAprovacao,iniciado,nota,profissional,identificadorUnico
0,REPROVADO,0,Ia para educação,75,6,false,0,true,276365396
0,CURSANDO,80,Metodologias Ágeis para Desenvolvimento de Sof...,75,6,true,8,false,272662909
0,AUSENTE,0,Power BI para Educação,,,false,,,735969834
1,REPROVADO,0,Ia para educação,,,false,,,735969834
0,CURSANDO,0,Metodologias Ágeis para Desenvolvimento de Sof...,75,6,false,0,false,709133461
...,...,...,...,...,...,...,...,...,...
0,CURSANDO,60,Introdução à Acessibilidade na Web,75,6,true,4.400000095367432,false,272138310
0,REPROVADO,25,Ia para educação,75,6,true,2.5,false,735972815
0,APROVADO,100,Ia para educação,75,6,true,9.5,true,272664718
1,CURSANDO,23,Introdução a Python,75,6,true,2.299999952316284,true,272664718


In [51]:
df_ind_reduzida = df_ind.drop(['tipo','chatId','data','nomeAluno','telefoneAluno','pk','empresa'], axis=1)
df_ind_reduzida

,idAluno,categoria,interativo
0,735975300,FINALIZAÇÃO CURSO,
1,743637878,PRAZO E POUCO DESEMPENHO,true
2,743637139,PRAZO E POUCO DESEMPENHO,true
3,703037977,PRAZO E POUCO DESEMPENHO,true
4,743637147,PRAZO E POUCO DESEMPENHO,true
...,...,...,...
10080,743637552,FINALIZAÇÃO CURSO,true
10081,735975850,FINALIZAÇÃO CURSO,true
10082,703037527,NÃO INICIOU / 3º CONTATO,true
10083,703038221,FINALIZAÇÃO CURSO,true


In [52]:
df_roi_reduzida = df_roi.drop(['tipo','chatId','data','nomeAluno','telefoneAluno','pk','empresa','efetivo'], axis=1)
df_roi_reduzida = df_roi_reduzida.reindex(columns = ['idAluno','categoria','interativo'])
df_roi_reduzida

,idAluno,categoria,interativo
0,272138310,FINALIZAÇÃO CURSO,false
1,272334954,INSCRITO E NÃO COMEÇOU,false
2,272334954,NÃO INICIOU / 3º CONTATO,false
3,272334954,INSCRITO E NÃO COMEÇOU,false
4,272334954,PRAZO E POUCO DESEMPENHO,false
...,...,...,...
2095,745406495,ALUNO TORNOU-SE AUSENTE,false
2096,745406495,ALUNO TORNOU-SE AUSENTE,false
2097,745406495,ALUNO À 10 SEM PROGRESSO,false
2098,745406515,ALUNO TORNOU-SE AUSENTE,false


### Criação de Novas Tabelas

In [53]:
df_ind_roi = pd.concat([df_ind_reduzida,df_roi_reduzida], axis=0)
df_ind_roi

,idAluno,categoria,interativo
0,735975300,FINALIZAÇÃO CURSO,
1,743637878,PRAZO E POUCO DESEMPENHO,true
2,743637139,PRAZO E POUCO DESEMPENHO,true
3,703037977,PRAZO E POUCO DESEMPENHO,true
4,743637147,PRAZO E POUCO DESEMPENHO,true
...,...,...,...
2095,745406495,ALUNO TORNOU-SE AUSENTE,false
2096,745406495,ALUNO TORNOU-SE AUSENTE,false
2097,745406495,ALUNO À 10 SEM PROGRESSO,false
2098,745406515,ALUNO TORNOU-SE AUSENTE,false


In [54]:
ddf_ind_roi = uni_count(df_ind_roi)
ddf_ind_roi

,idAluno,categoria,interativo
count,12185,12185,12185
unique,2623,8,3


In [111]:

individuos_unicos = df_ind_roi['idAluno'].unique()
df_atividade = pd.DataFrame({'idAluno': individuos_unicos})

categorias = df_ind_roi['categoria'].unique()
parametros = categorias
for y in parametros:
    bloco = []
    for x in df_atividade['idAluno'].values:
        contagem=np.sum(((df_ind_roi['idAluno']==x) & (df_ind_roi['categoria'] ==y )).astype(int))
        bloco.append(contagem)
    df_atividade[y] = bloco

In [118]:

parametros = 'true'

bloco = []
for x in df_atividade['idAluno'].values:
    contagem=np.sum(((df_ind_roi['idAluno']==x) & (df_ind_roi['interativo'] ==parametros )).astype(int))
    bloco.append(contagem)
df_atividade['interativo'] = bloco

In [119]:
df_atividade

,idAluno,FINALIZAÇÃO CURSO,PRAZO E POUCO DESEMPENHO,5 DIAS TERMINO E NÃO APROVADO,INSCRITO E NÃO COMEÇOU,BOAS-VINDAS,ALUNO TORNOU-SE AUSENTE,NÃO INICIOU / 3º CONTATO,ALUNO À 10 SEM PROGRESSO,interativo
0,735975300,2,0,1,0,0,0,0,0,2
1,743637878,1,3,0,2,1,0,2,0,5
2,743637139,1,1,0,0,1,0,1,0,3
3,703037977,1,1,0,0,1,0,1,0,3
4,743637147,1,1,0,0,1,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...
2618,743637922,0,0,0,0,1,0,0,0,0
2619,735973844,0,0,1,0,0,0,0,0,1
2620,735972901,0,0,0,0,0,1,0,0,1
2621,743638213,0,0,0,0,1,0,0,0,0


In [120]:
ddf_atv = uni_count(df_atividade)
ddf_atv

,idAluno,FINALIZAÇÃO CURSO,PRAZO E POUCO DESEMPENHO,5 DIAS TERMINO E NÃO APROVADO,INSCRITO E NÃO COMEÇOU,BOAS-VINDAS,ALUNO TORNOU-SE AUSENTE,NÃO INICIOU / 3º CONTATO,ALUNO À 10 SEM PROGRESSO,interativo
count,2623,2623.0,2623.0,2623.0,2623.0,2623.0,2623.0,2623.0,2623.0,2623.0
unique,2623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Escolhas finais

In [125]:
df_alu_reduzida.to_json('alunos_reduzida.json', indent=2, orient='records')


In [126]:
df_atividade.to_json('atividade.json',indent=2, orient='records')